# Pointcloud Tree Analysis
An experimental notebook

We will compare and analyse isolated trees using three different point cloud sources:
1. _[AHN](https://www.ahn.nl) data_ (publicly available)
2. _Cyclo Media_
3. _Sonarski_

A visual impression of what the an isolated three looks like in the three datasets. 
This particular tree is located at 1219.13, 4874.34 (LTR: AHN, CycloMedia, Sonarski):

![Comparison of datasets (side-view)](../imgs/20_side.png)
![Comparison of datasets (top-view)](../imgs/20_top.png)

#### Import Modules

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull

import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.las_utils as las_utils

#### Load datasets

In [ ]:
ahn_tree_path = '../datasets/single_selection/single_121913_487434_AHN.las'
cyclo_tree_path = '../datasets/single_selection/single_121913_487434_Cyclo.las'
sonarski_tree_path = '../datasets/single_selection/single_121913_487434_Sonarski.las'

In [ ]:
las_files = {
    'AHN': las_utils.read_las(ahn_tree_path), 
    'CycloMedia': las_utils.read_las(cyclo_tree_path),
    'Sonarski': las_utils.read_las(sonarski_tree_path)
}

## 1. Dataset statistics

In [ ]:
las_utils.print_statistics(las_files['CycloMedia'])

#### Open3D experiments

In [ ]:
cyclo_pcd = las_utils.to_o3d(las_files['CycloMedia'])
cyclo_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamRadius(.15))
cyclo_pcd

In [ ]:
# Outlier Selection
def outlier_removal(pcd):
    print("Radius oulier removal")
    cl, ind = pcd.remove_radius_outlier(nb_points=2, radius=0.3)

    inlier_cloud = pcd.select_by_index(ind)
    outlier_cloud = pcd.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

outlier_removal(cyclo_pcd)

In [ ]:
# Convex Hull
def pcd_convex_hull(pcd):
    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((1, 0, 0))
    o3d.visualization.draw_geometries([pcd, hull_ls])

pcd_convex_hull(cyclo_pcd)

![Convex Hull](../imgs/convex_hull.png)

In [ ]:
# Poisson Surface Reconstruction
def poisson_sr(pcd):
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8)
    o3d.visualization.draw_geometries([mesh])

poisson_sr(cyclo_pcd)

## Feature Extraction

**Tree height**

In [ ]:
# Option 1
# Substract tree pointcloud max and min.
def tree_height(las):
    height = las.xyz[:,2].max() - las.xyz[:,2].min()
    return height

# Option 2: 
# Compare tree cloud max with AHN (maaiveld) 
def tree_height_ahn(las, ahn):
    ground_z = None # TODO: z value at root.
    tree_max_z = las.xyz[:,2].max()
    height = tree_max_z - ground_z

**Base height**

Two options:
1. Substract tree pointcloud max and min.
2. Compare tree cloud max with AHN (maaiveld)

In [ ]:
def pcd_to_grid(pcd, bin_size=.1):
        points = np.asarray(pcd.points)
        min_x, max_x = min(points[:, 0])-.5*bin_size, max(points[:, 0])+.5*bin_size
        min_y, max_y = min(points[:, 1])-.5*bin_size, max(points[:, 1])+.5*bin_size
        min_z, max_z = min(points[:, 2])-.5*bin_size, max(points[:, 2])+.5*bin_size
        dimx = max_x - min_x
        dimy = max_y - min_y
        dimz = max_z - min_z
        bins = [np.uint(dimx/bin_size), np.uint(dimy/bin_size), np.uint(dimz/bin_size)]
        hist_range = [[min_x, max_x], [min_y, max_y], [min_z, max_z]]
        
        counts, edges = np.histogramdd(points, range=hist_range, bins=bins)
        origin = (hist_range[0][0]+bin_size/2,hist_range[1][0]+bin_size/2, hist_range[2][0]+bin_size/2)
        grid = counts > 0

        return grid, edges, bins, hist_range, origin

In [ ]:
grid = pcd_to_grid(cyclo_pcd)[0]
plt.imshow(np.flip(np.sum(grid, axis=1).T>0))

In [ ]:
# TODO: prone to hanging branches 

pcd = cyclo_pcd

bin_size = 0.15
points = np.asarray(pcd.points)
min_z, max_z = min(points[:, 2])-bin_size, max(points[:, 2])+bin_size
bins = np.arange(min_z, max_z, 0.1)

slice_ind = np.digitize(points[:,2], bins, right=True)

data = []
hulls = []
for i in np.unique(slice_ind):
    slice_mask = slice_ind==i
    if np.sum(slice_mask) > 3:
        slice_pts = points[slice_mask, :2]
        hull_pts = slice_pts[ConvexHull(slice_pts).vertices]
        hull_poly = Polygon(hull_pts)
        omtrek = hull_poly.length
        z = bins[i] - .5*bin_size
        data.append((z, omtrek))
        hulls.append(hull_poly)

base_i = np.argmax(np.array(data)[1:,1] - np.array(data)[:-1,1]  > .5)+1
plt.barh(np.array(data)[:base_i,0], np.array(data)[:base_i,1], height=0.05, color='brown')
plt.barh(np.array(data)[base_i:,0], np.array(data)[base_i:,1], height=0.05, color='green')
plt.ylabel('z-value')
plt.xlabel('Omtrek (m)')

base_height = data[base_i][0] - min(points[:, 2])
print('Base height:', np.round(base_height,2))

## Stem Features

In [ ]:
from skimage.measure import CircleModel, ransac
from sklearn.neighbors import NearestNeighbors

In [ ]:
def rodrigues_rot(points, vector1, vector2):
    """RODRIGUES ROTATION
    - Rotate given points based on a starting and ending vector
    - Axis k and angle of rotation theta given by vectors n0,n1
    P_rot = P*cos(theta) + (k x P)*sin(theta) + k*<k,P>*(1-cos(theta))"""

    if points.ndim == 1:
        points = points[np.newaxis, :]

    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)
    k = np.cross(vector1, vector2)
    if np.sum(k) != 0:
        k = k / np.linalg.norm(k)
    theta = np.arccos(np.dot(vector1, vector2))

    # MATRIX MULTIPLICATION
    P_rot = np.zeros((len(points), 3))
    for i in range(len(points)):
        P_rot[i] = (
            points[i] * np.cos(theta)
            + np.cross(k, points[i]) * np.sin(theta)
            + k * np.dot(k, points[i]) * (1 - np.cos(theta))
        )
    return P_rot

def circumferential_completeness_index(fitted_circle_centre, estimated_radius, slice_points):
    """
    Computes the Circumferential Completeness Index (CCI) of a fitted circle.
    Args:
        fitted_circle_centre: x, y coords of the circle centre
        estimated_radius: circle radius
        slice_points: the points the circle was fitted to
    Returns:
        CCI
    """

    sector_angle = 4.5  # degrees
    num_sections = int(np.ceil(360 / sector_angle))
    sectors = np.linspace(-180, 180, num=num_sections, endpoint=False)

    centre_vectors = slice_points[:, :2] - fitted_circle_centre
    norms = np.linalg.norm(centre_vectors, axis=1)

    centre_vectors = centre_vectors / np.atleast_2d(norms).T
    centre_vectors = centre_vectors[
        np.logical_and(norms >= 0.8 * estimated_radius, norms <= 1.2 * estimated_radius)
    ]

    sector_vectors = np.vstack((np.cos(sectors), np.sin(sectors))).T
    CCI = (
        np.sum(
            [
                np.any(
                    np.degrees(
                        np.arccos(
                            np.clip(np.einsum("ij,ij->i", np.atleast_2d(sector_vector), centre_vectors), -1, 1)
                        )
                    )
                    < sector_angle / 2
                )
                for sector_vector in sector_vectors
            ]
        )
        / num_sections
    )

    return CCI

def fit_circle_2D(points):

    CCI = 0
    r = 0
    xc, yc = np.mean(points[:, :2], axis=0)

    # Fit circle in new 2D coords with RANSAC
    if points.shape[0] >= 20:

        model_robust, inliers = ransac(
            points[:, :2],
            CircleModel,
            min_samples=int(points.shape[0] * 0.3),
            residual_threshold=0.05,
            max_trials=1000,
        )
        xc, yc = model_robust.params[0:2]
        r = model_robust.params[2]
        CCI = circumferential_completeness_index([xc, yc], r, points[:, :2])

    if CCI < 0.3:
        r = 0
        xc, yc = np.mean(points[:, :2], axis=0)
        CCI = 0

    return CCI, r, xc, yc

def fit_circle_3D(points, V):
    """
    Fits a circle using Random Sample Consensus (RANSAC) to a set of points in a plane perpendicular to vector V.
    Args:
        points: Set of points to fit a circle to using RANSAC.
        V: Axial vector of the cylinder you're fitting.
    Returns:
        cyl_output: numpy array of the format [[x, y, z, x_norm, y_norm, z_norm, radius, CCI, 0, 0, 0, 0, 0, 0]]
    """

    P = points[:, :3]
    P_mean = np.mean(P, axis=0)
    P_centered = P - P_mean
    normal = V / np.linalg.norm(V)
    if normal[2] < 0:  # if normal vector is pointing down, flip it around the other way.
        normal = normal * -1

    # Project points to coords X-Y in 2D plane
    P_xy = rodrigues_rot(P_centered, normal, [0, 0, 1])

    CCI, r, xc, yc = fit_circle_2D(P_xy)

    # Transform circle center back to 3D coords
    cyl_centre = rodrigues_rot(np.array([[xc, yc, 0]]), [0, 0, 1], normal) + P_mean
    cyl_output = np.array(
        [
            [
                cyl_centre[0, 0],
                cyl_centre[0, 1],
                cyl_centre[0, 2],
                normal[0],
                normal[1],
                normal[2],
                r,
                CCI
            ]
        ]
    )
    return cyl_output

def fit_cylinder(skeleton_points, point_cloud, num_neighbours):
    """
    Fits a 3D line to the skeleton points cluster provided.
    Uses this line as the major axis/axial vector of the cylinder to be fitted.
    Fits a series of circles perpendicular to this axis to the point cloud of this particular stem segment.
    Args:
        skeleton_points: A single cluster of skeleton points which should represent a segment of a tree/branch.
        point_cloud: The cluster of points belonging to the segment of the branch.
        num_neighbours: The number of skeleton points to use for fitting each circle in the segment. lower numbers
                        have fewer points to fit a circle to, but higher numbers are negatively affected by curved
                        branches. Recommend leaving this as it is.
    Returns:
        cyl_array: a numpy array based representation of the fitted circles/cylinders.
    """

    point_cloud = point_cloud[:, :3]
    skeleton_points = skeleton_points[:, :3]
    cyl_array = np.zeros((0, 8))
    line_centre = np.mean(skeleton_points[:, :3], axis=0)
    _, _, vh = np.linalg.svd(line_centre - skeleton_points)
    line_v_hat = vh[0] / np.linalg.norm(vh[0])

    if skeleton_points.shape[0] <= num_neighbours:
        group = skeleton_points
        line_centre = np.mean([np.min(group[:, :3], axis=0), np.max(group[:, :3], axis=0)], axis=0)
        length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
        plane_slice = point_cloud[
            np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
        ]  # calculate distances to plane at centre of line.
        if plane_slice.shape[0] > 0:
            cylinder = fit_circle_3D(plane_slice, line_v_hat)
            cyl_array = np.vstack((cyl_array, cylinder))
    else:
        for i in tqdm(range(skeleton_points.shape[0])):
            if skeleton_points.shape[0] > num_neighbours:
                nn = NearestNeighbors()
                nn.fit(skeleton_points)
                starting_point = np.atleast_2d(skeleton_points[np.argmin(skeleton_points[:, 2])])
                group = skeleton_points[nn.kneighbors(starting_point, n_neighbors=num_neighbours)[1][0]]
                line_centre = np.mean(group[:, :3], axis=0)
                length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
                plane_slice = point_cloud[
                    np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
                ]  # calculate distances to plane at centre of line.
                if plane_slice.shape[0] > 0:
                    cylinder = fit_circle_3D(plane_slice, line_v_hat)
                    cyl_array = np.vstack((cyl_array, cylinder))
                skeleton_points = np.delete(skeleton_points, np.argmin(skeleton_points[:, 2]), axis=0)
    return cyl_array
    

In [ ]:
def slice_tree(pcd, slice_thickness=0.15):

    points = np.asarray(pcd.points)
    min_z, max_z = min(points[:, 2])-slice_thickness, max(points[:, 2])+slice_thickness
    bins = np.arange(min_z, max_z, 0.1)
    slice_ind = np.digitize(points[:,2], bins, right=True)

    skeleton_points = np.zeros((0, 3))
    stem_points = np.zeros((0, 3))
    foliage_points = np.zeros((0, 3))
    for i in np.unique(slice_ind):
        new_slice = points[slice_ind==i]
        if i < 39: 
            if new_slice.shape[0] > 0:
                median = np.median(new_slice, axis=0)
                skeleton_points = np.vstack((skeleton_points, median))
                stem_points = np.vstack((stem_points, new_slice))
        else:
            foliage_points = np.vstack((foliage_points, new_slice))

        if i == 15:
            s_slice = (new_slice, median)

    return skeleton_points, stem_points, foliage_points, s_slice
    

In [ ]:
skeleton_points, stem_points, foliage_points, s_slice = slice_tree(cyclo_pcd)

In [ ]:
results = fit_cylinder(skeleton_points, stem_points, 2)

In [ ]:
fit_circle_3D(s_slice[0], np.array([0,0,1]))

In [ ]:
pcds= []

for result in results:
    centre = result[:3]
    normal = result[3:6]
    r = result[6]
    CCI = result[7]

    line_points = np.zeros((0,3))
    for i in range(18):
        phi = i*np.pi/9
        line_points = np.vstack((line_points, (centre[0] + r*np.cos(phi), centre[1] + r*np.sin(phi), centre[2])))

    lines = [(i,i+1) for i in range(17)]
    colors = [[0, 0, 0] for i in range(len(lines))]

    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(line_points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    pcds.append(line_set)

foliage = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(foliage_points))
foliage = foliage.paint_uniform_color([0.1,.6,0.1])
pcds.append(foliage)

stem = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(stem_points))
stem = stem.paint_uniform_color([0.5,0.3,0.05])
pcds.append(stem)

o3d.visualization.draw_geometries(pcds)


#### Branch split

In [ ]:
from plyfile import PlyData, PlyElement

In [ ]:
# Read ply tree skeleton file
plydata = PlyData.read('./tree7_skeleton.ply')

# Convert to points
vertices = np.array([[c for c in p] for p in plydata['vertex'].data])
new_vertices, reverse_ = np.unique(vertices, axis=0, return_inverse=True)
new_edges = np.array([reverse_[edge[0]] for edge in plydata['edge'].data])

# Branch Split Indices
u, c = np.unique(new_edges[:,0], return_counts=True)
branchsplit_ind = u[c > 1]

In [ ]:
skeleton_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(new_vertices))
skeleton_pcd = skeleton_pcd.paint_uniform_color([0.5,0.5,0.5])

sp_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(new_vertices[branchsplit_ind]))
sp_pcd = sp_pcd.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([skeleton_pcd,sp_pcd])

In [ ]:
import networkx as nx

def create_graph(vertices, edges):

    graph = nx.DiGraph()

    for i, vertex in enumerate(new_vertices):
        graph.add_node(i, x=vertex[0],y=vertex[1],z=vertex[2])

    for i,j in new_edges:
        graph.add_edge(i, j)

    return graph

def branch_path(graph, start):

    path = [start]
    while graph.out_degree(path[-1]) == 1:
        for node in graph.successors(path[-1]):
            path.append(node)
            break

    print(path[0], '-->', path[-1], '(nodes:',len(path),')')
    return path

# def get_first_split(graph):
    

In [ ]:
graph = create_graph(new_vertices, new_edges)

In [ ]:
# get stem till first split
start = np.argmin(new_vertices[:,2])
stem_path = branch_path(graph, start)

In [ ]:
mask = np.ones(graph.order(), dtype=bool)
mask[stem_path] = False
lowest_branch_node = np.where(mask)[0][np.argmin(new_vertices[mask,2])]
path_to_stem = nx.shortest_path(graph, source=stem_path[-1], target=lowest_branch_node)
path_to_stem

In [ ]:
branch = list(nx.nodes(nx.dfs_tree(graph, 6614)))

In [ ]:
skeleton_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(new_vertices))

tree_pcd = skeleton_pcd.select_by_index(branch+path_to_stem, invert=True)
tree_pcd = tree_pcd.paint_uniform_color([.5,.5,1])

brach_pcd = skeleton_pcd.select_by_index(branch)
brach_pcd = brach_pcd.paint_uniform_color([1,.8,.4])

to_stem_pcd = skeleton_pcd.select_by_index(path_to_stem)
to_stem_pcd = to_stem_pcd.paint_uniform_color([.2,.2,.5])

o3d.visualization.draw_geometries([tree_pcd,brach_pcd, to_stem_pcd])

In [ ]:
# NEXT STEP

# Lowest branch after each segment.

In [ ]:
skeleton_points = new_vertices[stem_path]